# Defining Problem Instance

In [72]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import EfficientSU2, TwoLocal, ZZFeatureMap
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit.result import QuasiDistribution
from qiskit.transpiler.passes import Decompose
from qiskit.quantum_info import Operator

from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit_algorithms import VQE, NumPyMinimumEigensolver, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA, GradientDescent, Optimizer, Minimizer
from qiskit_algorithms.gradients import  ParamShiftEstimatorGradient #, FiniteDiffEstimatorGradient, LinCombEstimatorGradient, QFI, DerivativeType, LinCombQGT

from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider, YahooDataProvider

from qiskit_algorithms import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.minimum_eigensolvers import MinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit.primitives import BackendSampler
from qiskit.primitives import Estimator
from qiskit_aer.primitives import Sampler

from scipy.optimize import minimize
from docplex.mp.model import Model
import numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings

In [73]:
# Filters warnings into critical and non-crital
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Necessary for tracking convergence
estimator = Estimator()
optimizer = COBYLA()
converter = QuadraticProgramToQubo()

# Gradient decsent methods
grad_pshift = ParamShiftEstimatorGradient(estimator) 

In [74]:
# Set random seed for reproducibility
np.random.seed(42)

# Number of properties
n_properties = 10

# Generate random expected returns (premiums) for properties
mu = np.random.uniform(0.05, 0.15, n_properties)

# Generate random correlation matrix and ensure it's symmetric and positive semi-definite
random_matrix = np.random.rand(n_properties, n_properties)
correlation_matrix = (random_matrix + random_matrix.T) / 2
np.fill_diagonal(correlation_matrix, 1)

# Generate covariance matrix by scaling the correlation matrix
sigma = correlation_matrix * np.outer(mu, mu)

# set risk factor
risk_factor = 0.5  

# Budget (30% of properties)
budget = int(0.3 * n_properties)

# # Convert to uint8

# # 1) For mu in [0.05, 0.15], multiplying by 255 puts values roughly in [12.75, 38.25].
# mu_scaled = mu * 255
# mu_clipped = np.clip(mu_scaled, 0, 255)
# mu_uint8 = mu_clipped.astype(np.uint8)

# # 2) For sigma = correlation_matrix * np.outer(mu, mu), values could be small.
# #    We'll similarly scale by 255, then clip and cast.
# sigma_scaled = sigma * 255
# sigma_clipped = np.clip(sigma_scaled, 0, 255)
# sigma_uint8 = sigma_clipped.astype(np.uint8)
# print(mu_uint8.dtype)
# print(sigma_uint8.dtype)



In [83]:
SU2_linear = EfficientSU2(n_properties, entanglement='linear', reps=1)

# Initialize parameters for the ansatz
initial_point = np.random.random(SU2_linear.num_parameters)

qp = PortfolioOptimization(expected_returns=mu, covariances=sigma, risk_factor=risk_factor, budget=budget).to_quadratic_program()
qubo = converter.convert(qp)

# Create the VQE instance
vqe = VQE(
    estimator=estimator,
    ansatz=SU2_linear,
    optimizer=optimizer,
    initial_point=initial_point
)

# Create the optimizer directly with VQE
optimizer = MinimumEigenOptimizer(min_eigen_solver=vqe)
result = optimizer.solve(qubo)

# Print the optimal portfolio based on budget and risk 
selection = result.x
value = result.fval
print("Optimal: selection {}, value {:.4f}".format(selection, value))

eigenstate = result.min_eigen_solver_result.eigenstate
probabilities = (
    eigenstate.binary_probabilities()
    if isinstance(eigenstate, QuasiDistribution)
    else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
)

# Prints all the options for portfolio and their value
print("\n----------------- Full result ---------------------")
print("This is a list of all possible portfolio options and their valuation:")
print("\n\t\t selection\t\t\t\t value\t\tprobability")
print("---------------------------------------------------")
probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

for k, v in probabilities:
    x = np.array([int(i) for i in list(reversed(k))])
    value = qp.objective.evaluate(x)
    print("%10s\t%.4f\t\t%.4f" % (x, value, v))

TypeError: MinimumEigenOptimizer supports qiskit.algorithms.minimum_eigensolvers.SamplingMinimumEigensolver, qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver, and qiskit.algorithms.minimum_eigen_solvers.MinimumEigensolver. But <class 'qiskit_algorithms.minimum_eigensolvers.vqe.VQE'> is given.